In [1]:
# !pip install tweeterpy
# !pip install gspread_dataframe 

In [2]:
# !jupyter nbconvert --to script "D:/Github/TweeterPy/get_tweet_stuff.ipynb" --no-prompt

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Markdown

In [4]:
import json 
from datetime import datetime
# from tweeterpy import TweeterPy
#twitter password  =AJ#~Z+cBUR&7r/ 

In [5]:
from gspread_dataframe import set_with_dataframe

In [6]:
import gspread

credentials = {
    "installed": {
        "client_id": "117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com",
        "project_id": "docs-382418",
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_secret": "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",
        "redirect_uris": ["http://localhost"],
    }
}

In [7]:
# gc, authorized_user = gspread.oauth_from_dict(credentials)

In [8]:
# twitter = TweeterPy()
# twitter.get_liked_tweets('rngland')
# twitter.get_friends('rngland', following=True)

In [9]:
with open('liked_tweets.py', 'r') as file:
    file_contents = file.read()
my_dict = {}
exec(file_contents, my_dict)

In [207]:
rest_ids = []
created_at = []
profile_images = []
screen_names = []
name = []
full_text = []
expanded_urls = []
binding_values = []


for i in range(len(my_dict['liked']['data'])):
    
    # rest_id
    if 'rest_id' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        rest_ids.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])
    
    # legacy
    # created_at
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        created_at.append(datetime.strptime(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'], "%a %b %d %H:%M:%S %z %Y").strftime("%Y-%m-%d %H:%M:%S")) 
        full_text.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['full_text']) 
        
    # core
    # profile_images    
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        profile_images.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']) 
    # screen_names
        screen_names.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name'])
    # name   
        name.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['name'])
    # binding_values
        if 'card' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
            binding_values.append((my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'],my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id']))  

# expanded_urls
for i in range(len(my_dict['liked']['data'])): 
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result'] and \
    len(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']) >= 1:
        _1 = my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']        
        _2 = [p['expanded_url'] for p in _1]
        expanded_urls.append('  '.join(_2))
    else:
        expanded_urls.append((' '))
     


p_img = ["=IMAGE(\"" + profile_images[i] + "\"" + ',4 ,45, 45)'  for i in range(len(profile_images))]

p_img = [image.replace("'", "") for image in p_img]

profile_url =['https://twitter.com/' + i for i in screen_names]

tweet_url =['https://twitter.com/' + screen_names[i] + '/status/' + rest_ids[i] for i in range(len(
screen_names))]




len(rest_ids)
len(created_at)
len(profile_url)len(p_img)
len(screen_names)
len(full_text)
len(tweet_url)
len(expanded_urls)
len(name)

664

664

664

664

664

664

664

668

664

In [355]:
expanded_urls = []

for i in range(len(my_dict['liked']['data'])): 
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result'] and \
    len(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']) >= 1:
        _1 = (my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls'], my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])
        _2 = ([p['expanded_url'] for p in _1[0]],_1[1])   
        expanded_urls.append(('  '.join(_2[0]),_2[1]))
    else:
        expanded_urls.append((' '))

        
          
        # _2 = [p['expanded_url'] for p in _1]
    #     expanded_urls.append('  '.join(_2))
    # else:
    #     expanded_urls.append((' '))
expanded_urls

[('https://news.tonydinh.com/p/my-solopreneur-story-zero-to-45kmo',
  '1705155679819084179'),
 ('https://python-pptx.readthedocs.io/en/latest/', '1700004016506216614'),
 ' ',
 ('https://github.com/deepfates/memery', '1704698971996803138'),
 ('https://openai.com/blog/red-teaming-network', '1704191384435990713'),
 ' ',
 ('https://hackerone.com/stripe?type=team', '1703819665162568007'),
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ('https://blog.janestreet.com/what-the-interns-have-wrought-2023/',
  '1701686506992083184'),
 ('https://go.nasa.gov/3sGKNLe', '1701236201489891515'),
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ('https://www.phoronix.com/review/windows11-wsl2-good',
  '1701420865039348103'),
 ' ',
 ('https://www.dipkumar.dev/becoming-the-unbeatable/posts/gpt-kvcache/',
  '1701322853419733320'),
 ('https://people.inf.ethz.ch/wirth/ProjectOberon/PO.System.pdf',
  '1701338500526678311'),
 ' ',
 ('https://jaykmody.com/blog/gpt-from-scratch/', '1701213155400253948'),


In [354]:
# _1 = (my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls'])
# _2 = [p['expanded_url'] for p in _1]
# expanded_urls.append('  '.join(_2))


# _1 = (my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls'], my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])  

# for i in _1[0]:
#     i['expanded_url']

_2 = ([p['expanded_url'] for p in _1[0]],_1[1])
_2

expanded_urls = []

expanded_urls.append(('  '.join(_2[0]),_2[1]))

expanded_urls


# for p in 


# [(p['expanded_url'],x) for x in _1 for p in x ]

        

# _2 = [(p['expanded_url']) for p in _1]



# _1
# _2

(['https://plottable.readthedocs.io/en/latest/',
  'https://github.com/znstrider/plottable'],
 '1588191925521326084')

[('https://plottable.readthedocs.io/en/latest/  https://github.com/znstrider/plottable',
  '1588191925521326084')]

In [10]:
rest_ids = []
for i in range(len(my_dict['liked']['data'])):
    if 'rest_id' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        rest_ids.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])


In [11]:
created_at = []
for i in range(len(my_dict['liked']['data'])):
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        created_at.append(datetime.strptime(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'], "%a %b %d %H:%M:%S %z %Y").strftime("%Y-%m-%d %H:%M:%S"))

In [12]:
profile_images = []
for i in range(len(my_dict['liked']['data'])):
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        profile_images.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https'])
        
len(profile_images)
profile_images[0:3]


664

['https://pbs.twimg.com/profile_images/1604292546645172224/SL57a00s_normal.png',
 'https://pbs.twimg.com/profile_images/1689852229736697856/aQmWl1ku_normal.jpg',
 'https://pbs.twimg.com/profile_images/1592599455895031808/VM5FWW47_normal.jpg']

In [13]:
p_img = ["=IMAGE(\"" + profile_images[i] + "\"" + ',4 ,45, 45)'  for i in range(len(profile_images))]

p_img = [image.replace("'", "") for image in p_img]

p_img[0:3]

['=IMAGE("https://pbs.twimg.com/profile_images/1604292546645172224/SL57a00s_normal.png",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1689852229736697856/aQmWl1ku_normal.jpg",4 ,45, 45)',
 '=IMAGE("https://pbs.twimg.com/profile_images/1592599455895031808/VM5FWW47_normal.jpg",4 ,45, 45)']

In [14]:
screen_names = []
for i in range(len(my_dict['liked']['data'])):
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        screen_names.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name'])

In [15]:
name = []
for i in range(len(my_dict['liked']['data'])):
    if 'core' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        name.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['name'])
# name       

In [16]:
profile_url =['https://twitter.com/' + i for i in screen_names]

In [17]:
full_text = []
for i in range(len(my_dict['liked']['data'])):
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        full_text.append(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['full_text'])           

In [18]:
tweet_url =['https://twitter.com/' + screen_names[i] + '/status/' + rest_ids[i] for i in range(len(
screen_names))]

In [182]:
expanded_urls = []

for i in range(len(my_dict['liked']['data'])): 
    if 'legacy' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result'] and \
    len(my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']) >= 1:
        _1 = my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']
        _2 = [p['expanded_url'] for p in urls]
        expanded_urls.append('  '.join(_2))
    else:
        expanded_urls.append((' '))


if len(rest_ids) != len(result): 
    while len(result) > len(rest_ids) and result[-1] == ' ': 
        _ = result.pop()



668

In [108]:
binding_values = []

for i in range(len(my_dict['liked']['data'])):
    if 'card' in my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']:
        binding_values.append((my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'],my_dict['liked']['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id']))   

In [159]:
card_title = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='title']

card_description = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='description']

card_thumbnail_image = [(p['value']['image_value']['url'],i[1]) for i in binding_values for p in i[0] if p['key'] =='thumbnail_image']

card_photo_image_full_size_alt_text = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='photo_image_full_size_alt_text']

In [172]:
field_names = [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"full_text",
"tweet_url",
"result",
"name",
# "card_title",
# "card_description",
# "card_thumbnail_image",
# "card_photo_image_full_size_alt_text"
]


data_dict = {field: eval(field) for field in field_names}

AssertionError: Must be list with elements of type string

In [154]:
no_id_fields =  [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"full_text",
"tweet_url",
"result",
"name",
]

def lists_to_df(no_id_lists, id_lists ):
    assert isinstance(no_id_lists, tuple), 'Must be list with elements of type string'
    assert isinstance(id_lists, tuple), 'Must be list with elements of type tuple'




,id,card_title
0,1705155679819084179,My solopreneur story: zero to $45K/mo in 2 years
1,1700004016506216614,NaN
2,1704989552459579398,NaN
3,1704698971996803138,GitHub - deepfates/memery: Search over large i...
4,1704191384435990713,NaN
...,...,...
659,1587867700231081987,NaN
660,1584659361632161792,NaN
661,1552124372231458817,NaN
662,1499838567111143431,NaN


In [64]:
field_names = [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"full_text",
"tweet_url",
"result",
"name",
# "card_title",
# "card_description",
# "card_thumbnail_image",
# "card_photo_image_full_size_alt_text"
]


data_dict = {field: eval(field) for field in field_names}

In [59]:

len(rest_ids)
len(created_at)
len(profile_url)
len(p_img)
len(screen_names)
len(full_text)
len(tweet_url)
len(result)
len(name)
len(card_title)
len(card_description)
len(card_thumbnail_image)
len(card_photo_image_full_size_alt_text)


664

664

664

664

664

...

664

124

119

91

36

In [66]:
import pandas as pd

df = pd.DataFrame(data_dict)
df
# df.to_csv('D:/Github/TweeterPy/liked_tweets.csv')

,rest_ids,created_at,profile_url,p_img,screen_names,full_text,tweet_url,result,name
0,1705155679819084179,2023-09-22 09:42:48,https://twitter.com/tdinh_me,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tdinh_me,My full story: \n\nhttps://t.co/SXBdF7Th2B,https://twitter.com/tdinh_me/status/1705155679...,https://news.tonydinh.com/p/my-solopreneur-sto...,Tony Dinh 🎯
1,1700004016506216614,2023-09-08 04:31:56,https://twitter.com/theJordanNoone,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",theJordanNoone,@ryxcommar meanwhile the best coders write the...,https://twitter.com/theJordanNoone/status/1700...,https://python-pptx.readthedocs.io/en/latest/,Jordan Noone 🇺🇸
2,1704989552459579398,2023-09-21 22:42:41,https://twitter.com/mayfer,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",mayfer,scammers salivating harder by the hour,https://twitter.com/mayfer/status/170498955245...,,murat 🍥
3,1704698971996803138,2023-09-21 03:28:01,https://twitter.com/cto_junior,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",cto_junior,@up_61347132 @sog_on_bird_app @deepfates https...,https://twitter.com/cto_junior/status/17046989...,https://github.com/deepfates/memery,TDM (e/flλ)
4,1704191384435990713,2023-09-19 17:51:03,https://twitter.com/gdb,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",gdb,"Get paid to red team our models, using your do...",https://twitter.com/gdb/status/170419138443599...,https://openai.com/blog/red-teaming-network,Greg Brockman
...,...,...,...,...,...,...,...,...,...
659,1587867700231081987,2022-11-02 18:02:15,https://twitter.com/NoahTopper,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",NoahTopper,I’m reading “What Computers Still Can’t Do” an...,https://twitter.com/NoahTopper/status/15878677...,https://youtu.be/5Ldn_nCPNNQ,Noah Topper 🔍 (in Seattle 5/6 - 9/29)
660,1584659361632161792,2022-10-24 21:33:28,https://twitter.com/radshaan,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",radshaan,Every single talented engineer (electrical/sof...,https://twitter.com/radshaan/status/1584659361...,,Ishan
661,1552124372231458817,2022-07-27 02:51:02,https://twitter.com/BenHanowell,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",BenHanowell,@PhDemetri @baursafi It’s most popular data ma...,https://twitter.com/BenHanowell/status/1552124...,https://capitalone.wd1.myworkdayjobs.com/Capit...,Ben Hanowell 🦜☕
662,1499838567111143431,2022-03-04 20:05:54,https://twitter.com/tyba_latam,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tyba_latam,"¡Invierte tu dinero, no juegues con él! tyba, ...",https://twitter.com/tyba_latam/status/14998385...,https://plottable.readthedocs.io/en/latest/ h...,tyba


In [ ]:
sh = gc.open("liked_tweets")
worksheet  = sh.get_worksheet(0)    

In [293]:
worksheet.format(f"g2:g{len(rest_ids)}", {
  "textFormat": {
    "fontSize": 14
   }
})
set_with_dataframe(worksheet, df) 

In [ ]:
print(my_dict['liked']['data'][8]['content']['itemContent']['tweet_results']['result']['legacy']['full_text'])

urls = my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']

for i in urls:
    i['expanded_url']

# [i for i in urls['expanded_url']]

screen_name = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name']

rest_id = my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['rest_id']

'https://twitter.com/' + screen_name + '/status/' + rest_id

screen_name
rest_id

print(my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['description'])


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['legacy']['created_at']

#  'in_reply_to_screen_name': 'ryxcommar',
# 'in_reply_to_status_id_str': '1700002950012145747',

# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['is_blue_verified']

my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']


my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][2]['value']['string_value']



# my_dict['liked']['data'][0]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values']

# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][4]
# my_dict['liked']['data'][3]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'][0]['value']['string_value']


# card  
    #   {'key': 'title',
    #        'value': {'string_value': 'My solopreneur story: zero to $45K/mo in 2 years',
    #   {'key': 'description',
    #        'value': {'string_value': 'Today is exactly 2 years since I quit my job and become a full-time indie hacker.',
        #   {'key': 'photo_image_full_size_alt_text',
        #    'value': {'string_value': 'Search over large image datasets with natural language and computer vision! - GitHub - deepfates/memery: Search over large image datasets with natural language and computer vision!',
        #      {'key': 'thumbnail_image',
        #    'value': {'image_value': {'height': 200,
        #      'width': 400,
        #      'url': 'https://pbs.twimg.com/card_img/1704513029398745088/aRyYEIOj?format=jpg&name=400x400'},
        #     'type': 'IMAGE'}},
        ##card 

I want to share my non-rigorous vibes based approach towards data science modeling of data. A lot of my job as a staff data scientist (i'm staff btw) is doing a vibe-check on technical problems, and helping other data scientists from going down an intractable path.
Indie hacker.

🧠 @TypingMindApp $30K/mo
📸 @XnapperHQ $6K/mo
🛠 @devutils_app $5.5K/mo
🪄 @blackmagic_so (Sold, was $14K/mo)

💬 https://t.co/9b1SSybSdG


'Today is exactly 2 years since I quit my job and become a full-time indie hacker.'